# Knowledge Structure Graph Generator (Neo4J Implementation)

## 1. DB연결

In [1]:
import pymysql
import pandas as pd
import re

In [2]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [3]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [5]:
result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
       DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0        hemp      scs  sourc,origin,1.0\nsourc,millennium,1.0\nsourc,...
1          감초      scs  함량,재배,2.9\n효능,역할,3.21\n하루,이상,3.9\n이상,단맛,3.9\n한...
2          녹차      scs  순수,티백,1.52\n제품,제조,2.15\n효과,흥분,2.95\n제품,순수,2.95...
3          대마      scs  마약,법률,3.06\n약물,중독,3.26\n한국,미국,3.87\n필로폰,헤로인,4....
4         바이오      scs  소재,개발,3.07\n분야,성과,3.6\n기술,식품,3.71\n분야,식품,3.79\...
5    인공지능_이루다      scs  제공,특정,2.76\n대화,연인들,3.54\n개발사,여성,3.54\n말투,학습,4....
6   인공지능_이루다2      scs  특정,자사,1.63\n수집,변호사,2.1\n개인정보,유출,2.76\n개인정보,수집,...
7     인공지능_제조      scs  제조,역할,1.0\n제조,육성,1.0\n역할,육성,1.0\n산업,제조,1.34\n산...
8     인공지능_호텔      scs  호텔,서비스,1.99\n코로나,업계,2.76\n제공,정보,2.98\n제공,차량,2....
9           차      scs  남부,지방,2.71\n재배,지역,4.03\n식물,우려,4.81\n남부,지역,4.85...
10          콩      scs  두부,된장,1.8\n고기,짜파게티,2.53\n여성,에스트로겐,3.0\n성분,여성호르...
11         홍차      scs  여성,커피,2.96\n시간,우

In [6]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [7]:
file_name = "바이오"

#### 추출하기 원하는 인덱스 선정

In [8]:
data = knowledge_structure[4] #인덱스만 변경하기

In [9]:
contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [10]:
#노드 추가
edge_list = list()
node_set = set()

for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    edge_list.append(value) # 이중 리스트
    node_set.add(value[0]) # 노드들을 셋에 추가하여 유니크 값만 저장
    node_set.add(value[1]) 

## 3. Neo4J 연결

In [11]:
from py2neo import Graph, Node, Relationship

In [13]:
graph = Graph('bolt://neo4j@localhost:7687', auth = ("neo4j", "1234"))

# Authentification Error가 날 경우, 데이터베이스 생성시 설정한 name과 password를 써줍니다.
# graph = Graph('bolt://neo4j@localhost:7687', auth = ("neo4j", "password"))

graph

Graph('bolt://neo4j@localhost:7687')

In [14]:
node2idx = dict() # 각 노드에 인덱스 부여
for idx, node in enumerate(node_set):
    node2idx[node] = idx # 노드가 key, 인덱스가 value
node2idx

{'식품': 0,
 '성과': 1,
 '생산': 2,
 '우수': 3,
 '분야': 4,
 '소재': 5,
 '특성': 6,
 '산업': 7,
 '기술': 8,
 '해수부': 9,
 '개발': 10,
 '연구': 11,
 '바이오': 12,
 '기업': 13,
 '지원': 14,
 '정보': 15,
 '연구개발': 16,
 '인프라': 17,
 '제품': 18,
 '시장': 19}

In [15]:
nodes_neo = [Node("Keyword", name = node) for node in node2idx.keys()] # Py2neo를 이용해 Node를 선언. Label은 Keyword, Property는 name이다.
nodes_neo

[Node('Keyword', name='식품'),
 Node('Keyword', name='성과'),
 Node('Keyword', name='생산'),
 Node('Keyword', name='우수'),
 Node('Keyword', name='분야'),
 Node('Keyword', name='소재'),
 Node('Keyword', name='특성'),
 Node('Keyword', name='산업'),
 Node('Keyword', name='기술'),
 Node('Keyword', name='해수부'),
 Node('Keyword', name='개발'),
 Node('Keyword', name='연구'),
 Node('Keyword', name='바이오'),
 Node('Keyword', name='기업'),
 Node('Keyword', name='지원'),
 Node('Keyword', name='정보'),
 Node('Keyword', name='연구개발'),
 Node('Keyword', name='인프라'),
 Node('Keyword', name='제품'),
 Node('Keyword', name='시장')]

- 참고!
    - edge_list는 각 edge를 리스트로 한 이중리스트입니다.
    - 각 edge는 [**노드1, 노드2, 점수**]의 형태로 되어 있습니다.
    - Py2neo 문법 : Relationship(node1, relationship, node2, *relationship_property(옵션)*)
    

In [17]:
tx = graph.begin() # 트랜잭션 객체를 선언하여 시작합

for edge in edge_list: # 각 edge에 대해 관계를 선언
    node1 = nodes_neo[node2idx[edge[0]]]
    node2 = nodes_neo[node2idx[edge[1]]]
    re = Relationship(node1, edge[2], node2, name = "score")
    # 생성한 노드 nodes_neo는 위와 같이 node2id를 이용해 인덱스로 접근할 수 있습니다. 
    print(re)
    tx.create(re) # 선언해둔 노드나 관계 객체를 생성
tx.commit() # 트랜잭션을 commit하여 DB에 반영하며 종료됩니다. 선언했던 노드와 관계가 실제로 DB에 반영되는 시기입니다.

if tx.finished() == False:
    print("Transaction is still open")
else:
    print("Transaction has been completed")

(소재)-[:3.07 {name: 'score'}]->(개발)
(분야)-[:3.6 {name: 'score'}]->(성과)
(기술)-[:3.71 {name: 'score'}]->(식품)
(분야)-[:3.79 {name: 'score'}]->(식품)
(개발)-[:3.8 {name: 'score'}]->(기술)
(기술)-[:3.9 {name: 'score'}]->(우수)
(인프라)-[:3.9 {name: 'score'}]->(특성)
(지원)-[:3.98 {name: 'score'}]->(제품)
(소재)-[:4.0 {name: 'score'}]->(생산)
(소재)-[:4.14 {name: 'score'}]->(연구)
(바이오)-[:4.25 {name: 'score'}]->(산업)
(성과)-[:4.32 {name: 'score'}]->(특성)
(바이오)-[:4.42 {name: 'score'}]->(시장)
(연구개발)-[:4.43 {name: 'score'}]->(분야)
(소재)-[:4.55 {name: 'score'}]->(제품)
(제품)-[:4.55 {name: 'score'}]->(생산)
(지원)-[:4.66 {name: 'score'}]->(기업)
(기업)-[:4.66 {name: 'score'}]->(분야)
(바이오)-[:4.78 {name: 'score'}]->(인프라)
(지원)-[:4.78 {name: 'score'}]->(산업)
(소재)-[:4.81 {name: 'score'}]->(정보)
(산업)-[:4.81 {name: 'score'}]->(해수부)
(정보)-[:4.81 {name: 'score'}]->(생산)
Transaction has been completed


In [18]:
graph.run("MATCH(n) RETURN n")

 n                                         
-------------------------------------------
 (_0:Keyword {name: '\ub300\ud654'})       
 (_1:Keyword {name: '\uc5f0\uc778\ub4e4'}) 
 (_2:Keyword {name: '\uac1c\ubc1c\uc0ac'}) 

* 참고
    - run의 경우, 수행과 동시에 자동으로 commit되어 따로 트랜잭션 객체를 생성하거나 commit할 필요가 없습니다.

---

### Neo4J로 생성된 지식구조 확인
이제 다시 Neo4J Browser로 가서 `MATCH(n) RETURN n` 쿼리를 입력하면 그래프 형태로 확인할 수 있습니다.

- Q. Neo4J Browser를 못 찾겠어요!
    - A. Neo4J 데스크탑에서 localhost 클릭하면 창이 새로 뜹니다.